# Evidence Ledger Builder

**Goal:** Build a signed ledger (`03_Evidence/hashes/ledger.csv`) for audit evidence without moving files.

In [8]:
from pathlib import Path
from datetime import datetime
import csv, hashlib
import pandas as pd

BASE = Path('..').resolve()
print(BASE)

ROOT   = BASE / '03_evidence'              
LEDGER = ROOT / "ledger.csv" 
print(ROOT)
print(LEDGER)


C:\Project\Miss Shoes
C:\Project\Miss Shoes\03_evidence
C:\Project\Miss Shoes\03_evidence\ledger.csv


In [ ]:
SIGNER = " Arjona_002 / GRC"    
ENVIRONMENT = "non-prod"
PII_FLAG_DEFAULT = "synthetic"

def ensure_header(csv_path: Path, header):
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    if not csv_path.exists():
        with open(csv_path, "w", newline="", encoding="utf-8") as f:
            csv.writer(f).writerow(header)

def sha256sum(p: Path) -> str:
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def artifact_type_from_path(p: Path) -> str:
    ext = p.suffix.lower().lstrip(".")
    if ext == "":
        return "file"
    return ext

def to_path(s: str) -> Path:
    p = Path(s)
    
    if not p.is_absolute():
       
        p = (ROOT.parent / p).resolve()
    return p


HEADER = ["artifact_path","artifact_type","control_id","run_id","timestamp_utc",
          "sha256","signer","environment","pii_flag","notes"]
ensure_header(LEDGER, HEADER)


ARTIFACTS = [
    '01_Case/A_Log_Detections/outputs/alerts_evidence.csv',
    '01_Case/B_Config_ChangeMgmt/outputs/config_drift_evidence.csv',
    '01_Case/C_Privacy_DSAR/outputs/dsar_log_evidence.csv',
    '01_Case/C_Privacy_DSAR/outputs/customers_DSAR-20250911-02.csv',
    '01_Case/C_Privacy_DSAR/outputs/customers_DSAR-20250911-05.csv',
    '01_Case/C_Privacy_DSAR/outputs/customers_DSAR-20250911-08.csv',
    '01_Case/C_Privacy_DSAR/outputs/delete_preview_DSAR-20250911-03.csv',
    '01_Case/C_Privacy_DSAR/outputs/delete_preview_DSAR-20250911-07.csv',
    '01_Case/C_Privacy_DSAR/outputs/dsar_DSAR-20250911-02.zip',
    '01_Case/C_Privacy_DSAR/outputs/dsar_DSAR-20250911-05.zip',
    '01_Case/C_Privacy_DSAR/outputs/dsar_DSAR-20250911-08.zip',
]


files = [to_path(s) for s in ARTIFACTS]


existing = set()
if LEDGER.exists():
    with open(LEDGER, "r", encoding="utf-8") as f:
        for row in csv.DictReader(f):
            existing.add((row.get("artifact_path",""), row.get("sha256","")))


rows_to_append = []
for p in files:
    if not p.exists():
        print(f"[WARN] Not found: {p}")
        continue
    rel = str(p.relative_to(ROOT.parent))  
    digest = sha256sum(p)
    if (rel, digest) in existing:
        print(f"[SKIP] Already recorded: {rel}  SHA256={digest[:8]}...")
        continue
  
    artifact_type = artifact_type_from_path(p)
    control_id = ""   
    run_id = datetime.now().strftime("%Y%m%dT%H%M%SZ")
    timestamp_utc = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    signer = SIGNER
    environment = ENVIRONMENT
    pii_flag = PII_FLAG_DEFAULT
    notes = ""
    row = {
        "artifact_path": rel,
        "artifact_type": artifact_type,
        "control_id": control_id,
        "run_id": run_id,
        "timestamp_utc": timestamp_utc,
        "sha256": digest,
        "signer": signer,
        "environment": environment,
        "pii_flag": pii_flag,
        "notes": notes,
    }
    rows_to_append.append(row)
    print(f"[ADD] {rel}  {artifact_type}  SHA256={digest[:8]}...")


if rows_to_append:
    with open(LEDGER, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=HEADER)
        for r in rows_to_append:
            w.writerow(r)
    print(f"\n{len(rows_to_append)} entries appended to ledger: {LEDGER}")
else:
    print("\nNo new artifacts to append.")


[ADD] 01_Case\A_Log_Detections\outputs\alerts_evidence.csv  csv  SHA256=c2da568f...
[ADD] 01_Case\B_Config_ChangeMgmt\outputs\config_drift_evidence.csv  csv  SHA256=7917588a...
[ADD] 01_Case\C_Privacy_DSAR\outputs\dsar_log_evidence.csv  csv  SHA256=6e481d3f...
[ADD] 01_Case\C_Privacy_DSAR\outputs\customers_DSAR-20250911-02.csv  csv  SHA256=6fa9b877...
[ADD] 01_Case\C_Privacy_DSAR\outputs\customers_DSAR-20250911-05.csv  csv  SHA256=eb32688c...
[ADD] 01_Case\C_Privacy_DSAR\outputs\customers_DSAR-20250911-08.csv  csv  SHA256=7e65790a...
[ADD] 01_Case\C_Privacy_DSAR\outputs\delete_preview_DSAR-20250911-03.csv  csv  SHA256=e242e602...
[ADD] 01_Case\C_Privacy_DSAR\outputs\delete_preview_DSAR-20250911-07.csv  csv  SHA256=e242e602...
[ADD] 01_Case\C_Privacy_DSAR\outputs\dsar_DSAR-20250911-02.zip  zip  SHA256=5e392ca2...
[ADD] 01_Case\C_Privacy_DSAR\outputs\dsar_DSAR-20250911-05.zip  zip  SHA256=455a7404...
[ADD] 01_Case\C_Privacy_DSAR\outputs\dsar_DSAR-20250911-08.zip  zip  SHA256=922ca933...


### Security note (public portfolio)
- Publish hashes only for **synthetic/non-sensitive** artifacts in public repos.
- Never publish secrets/PII.
